In [2]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from config import login
import csv

In [19]:
# Loading csv files from movies_csv

links = pd.read_csv("movies_csv/links.csv")
movies = pd.read_csv("movies_csv/movies.csv")
ratings = pd.read_csv("movies_csv/ratings.csv")
tags = pd.read_csv("movies_csv/tags.csv")

In [3]:
# Open .dat files from GroupLensResearch from tag_files

tag_file = open("tag_files/tags.dat")
movies_file = open("tag_files/movies.dat")
tag_relevance = open("tag_files/tag_relevance.dat")


In [4]:
# Formatting .dat files for pandas df

tag_df = pd.read_csv(tag_file, sep='\t', lineterminator='\n')
movies_df = pd.read_csv(movies_file, sep='\t', lineterminator='\n')
tag_relevance = pd.read_csv(tag_relevance, sep='\t', lineterminator='\n')
  
tag_nlp = tag_df.rename(columns={"0": "tag_id", "007": "tag", "61": "tag_popularity"})
movies_nlp = movies_df.rename(columns={"1": "movie_id", "Toy Story (1995)": "title", "53059": "movie_popularity"})
relevance_nlp = tag_relevance.rename(columns={"1": "movie_id", "0": "tag_id", "0.032": "relevance"}) 

In [16]:
# Narrowing down dataframe for size and relevance issues

relevance_nl = relevance_nlp.where(relevance_nlp["relevance"] >= 0.5)
relevant = pd.DataFrame(relevance_nl)
relevant = relevant.dropna()
relevant.head()
relevant.to_csv("tag_files/relevant.csv")

,movie_id,tag_id,relevance
9,1.0,10.0,0.563
27,1.0,28.0,0.894
28,1.0,29.0,0.699
59,1.0,60.0,0.623
60,1.0,61.0,0.655


In [24]:
relevant.to_csv("tag_files/relevant.csv")

In [17]:
# Establishing connection
# Make sure to manually create "movie_db" in postgres 

db_url = 'postgresql://' + login + '@localhost:5432/movie_db'
engine = create_engine(db_url)
connection = engine.connect()

In [20]:
# Renaming columns
links = links.rename(columns={"movieId": "movie_id"})
movies = movies.rename(columns={"movieId": "movie_id", "genres": "genre"})
ratings = ratings.rename(columns={"movieId": "movie_id", "userId": "user_id"})
tags = tags.rename(columns={"movieId": "movie_id", "userId": "user_id"})

In [21]:
# Adding csv's to postgres

links.to_sql("links", engine, if_exists='append', index=False)
movies.to_sql("movies", engine, if_exists='append', index=False)
ratings.to_sql("ratings", engine, if_exists='append', index=False)
tags.to_sql("tags", engine, if_exists='append', index=False)

In [ ]:
# Adding .dat (in pd.df format) in csv

tag_nlp.to_sql("tag_nlp", engine, if_exists='append', index=False)
movies_nlp.to_sql("movies_nlp", engine, if_exists='append', index=False)
relevant.to_sql("relevant", engine, if_exists='append', index=False)

In [23]:
# Make sure to close the connection

connection.close()

In [18]:
# Example query

avg_ = f"select r.movie_id, avg(rating), m.title, m.genre from ratings r \
inner join movies m ON r.movie_id=m.movie_id \
group by r.movie_id, m.title, m.genre \
order by avg desc;"

avg_rating = pd.read_sql(avg_, connection)